In [3]:
import torch
from transformers import pipeline

#pipe = pipeline("text-generation", model="HuggingFaceH4/starchat-beta", torch_dtype=torch.bfloat16, device_map="auto")
pipe = pipeline("text-generation", model="/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f", torch_dtype=torch.bfloat16, device_map="cpu")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of GPTBigCodeForCausalLM were not initialized from the model checkpoint at /data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# We use a variant of ChatML to format each message
import time

t1 = time.time()

prompt_template = "<|system|>\n<|end|>\n<|user|>\n{query}<|end|>\n<|assistant|>"
prompt = prompt_template.format(query="How do I sort a list in Python?")
print("before model start")
# We use a special <|end|> token with ID 49155 to denote ends of a turn
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.2, top_k=50, top_p=0.95, eos_token_id=49155)

t2 = time.time()
t2-t1

Setting `pad_token_id` to `eos_token_id`:49155 for open-end generation.


before model start


627.7025089263916

In [6]:
outputs[0]["generated_text"].split("\n")

['<|system|>',
 '<|end|>',
 '<|user|>',
 'How do I sort a list in Python?<|end|>',
 '<|assistant|>',
 'There are multiple ways to sort a list in Python. One of the most common ways is using the sort() method.',
 '',
 "Here's an example:",
 '',
 '```',
 'numbers = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]',
 'numbers.sort()',
 'print(numbers)',
 '```',
 '',
 'This will sort the list in place and print the sorted list.',
 '',
 'Another way to sort a list is using the sorted() function. This function returns a new sorted list without modifying the original list.',
 '',
 "Here's an example:",
 '',
 '```',
 'numbers = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]',
 'sorted_numbers = sorted(numbers)',
 'print(sorted_numbers)',
 '```',
 '',
 'In this example, the sorted() function is used to create a new sorted list, and the original list is not modified.',
 '',
 'There are also other sorting algorithms available in the built-in sort module, such as quicksort, heapsort, and merge sort. You can learn more about 

In [ ]:
#!pip install transformers==4.34.1

In [7]:
from refractml import *
from refractml.constants import MLModelFlavours

# # new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [12]:
class ScoreTemplateExample(ScoreBase):
#     """
#     This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
#     """    
     def __init__(self):
        super().__init__()
        
        self.model_loaded = None
        
        import torch
        from transformers import pipeline

        if self.model_loaded is None:
            print("LLM model loading from data section")
            self.model_loaded = pipeline("text-generation", model="/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f", torch_dtype=torch.bfloat16, device_map="auto")

     def request_processing_fn(self, request) :
#         """
#         Processes Request Object -> List[Input data, ..]. It could be:
#                A List Mapping of All Value Can Be one of : 
#                    - List[ [Feature_Value1, Feature_Value2, ...], [...] ]
#                    - List[numpy.array(), numpy.array(), ...]
#                    - List[tf.Tensor, tf.Tensor, tf.Tensor, ...]
#                    - List[ SingleSample, SingleSample]
                   
#         :return: (n_inputs, payload's)
        
#         Warnings:
#         1. Do not reshape your final output for single sample here, do it in prediction.
#            Else payloads will be invalidated for extraction at raw and extraction level.
#         """
         final_payload = []
         raw_payload = request.json["payload"]
         return (1, raw_payload) 
    
     def pre_processing_fn(self,payload):
#         # All preprocessing step must occur in this section
#         # Takes Single Sample -> Returns Single Sample
        
#         # Not Doing Any Preprocessing Hence Returned payload
#         print("payload is ", payload)
        
         return payload

     def prediction_fn(self,
                       model: Any,
                       input_query 
                       ):
#         """
#                 Does the main prediction on pre_processed_input(Single Sample) using supplied model .

#                 :param model: Supported Model
#                 :param pre_processed_input: Single Preprocessed Payload
#                 :return: Prediction Value From the model
                
#                 Important Notes:
#                 - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
#         """
#         model_loaded = self.model_loaded
#         mod = model_loaded
#         text = pre_processed_input #this is tuple we can iterate if there is number of input
        
          prompt_template = "<|system|>\n<|end|>\n<|user|>\n{query}<|end|>\n<|assistant|>"
          text = input_query
          prompt = prompt_template.format(query=text)
#         # We use a special <|end|> token with ID 49155 to denote ends of a turn
          outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.2, top_k=50, top_p=0.95, eos_token_id=49155)
          preds = str(outputs[0]["generated_text"].split("<|assistant|>")[1])
        
        
          print("prediction is \n: ",preds)
    
          return preds

     class Meta:    
         # List of Callables() can be attached For Calling After AnSd Before Scoring
         def __init__(self):
             self.name = "Pre Hooked Me !"
             self.pre_call_hooks.append(self.print_)
         def print_(self):
             print(self.name)
             pre_call_hooks = []
             post_call_hooks = []

In [13]:
score_ = ScoreTemplateExample()
import requests
req = requests.Request()

req.json = {"payload":"How do I sort a list in Python?"}

AttributeError: 'Meta' object has no attribute 'pre_call_hooks'

In [5]:
model_predictions = score_.score(model, req, dry_run=True)

NameError: name 'score_' is not defined